In [10]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer, execute
# Create a quantum circuit with 10 qubits (5 for the input values and 5 for the XOR results)
qr = QuantumRegister(12, 'q') # they are used to store the qubits, A qubit can have multiple values in it
cr = ClassicalRegister(10, 'c') # they are used to store the results when qubits are measured
circuit = QuantumCircuit(qr, cr)
# Fixed 5-bit string for XOR
fixed_value = "10101"  # Key
# Create lists to store XOR results and transposed results
xor_results = []
binary_value = []
transposed_results = []
transpose = 1

#Function to create array of binary string input
def create_binary_string():
    for i in range(32):
        x = format(i, '05b')
        binary_value.append(x)
    return binary_value

#permutation method
def permutation(i,n):
    for i in range(n):
        a = i + 1
        if a > n-1:
            a -=5
        b = i + 2
        if b > n-1:
            b -=5
        circuit.reset(qr[10])
        circuit.cx(qr[b], qr[i])
        circuit.ccx(qr[a], qr[b], qr[10])
        circuit.cx(qr[b], qr[i])
        circuit.cx(qr[10], qr[i])
        circuit.cx(qr[a], qr[i])
        
# Expolring Simon Problem E(x) + P(x)
def simon_problem(binary_s):
        for i in range(32):
            binary_value = binary_s[i]
            for j in range(5):
                     if binary_value[j] == '1':
                            circuit.x(qr[j]) 
            for j in range(5):
                    if fixed_value[j] == '1':
                            circuit.x(qr[j + 5]) 
            for j in range(5):
                       circuit.cx(qr[j], qr[j + 5])   # First XOR (x + k1)                 
            
            # Permutation after first Xor
            permutation(5,10)  #P(x+k1)
            
            for j in range(5):
                circuit.reset(qr[j])
            for j in range(5):     
                    if fixed_value[j] == '1':
                            circuit.x(qr[j])
            for j in range(5):
                circuit.cx(qr[j], qr[j+5]) #P(x+k1) + k2
            
            for j in range(5):
                circuit.reset(qr[j]) #To do P(x)
            circuit.reset(qr[10])
            circuit.reset(qr[11])
            #print(binary_value)
            for j in range(5):
                     if binary_value[j] == '1':
                            circuit.x(qr[j])
            permutation(0,5)
            for j in range(5):
                circuit.cx(qr[j], qr[j+5]) #P(x+k1) + k2 + P(x)
            
            for j in range(5):
                    circuit.measure(qr[j+5], cr[j])
            simulator = Aer.get_backend('qasm_simulator')
            compiled_circuit = transpile(circuit, simulator)
            job = execute(compiled_circuit, simulator, shots=1)
            result = job.result()
            counts = result.get_counts()
            xor_result = list(counts.keys())[0][::-1][:5]  # Extract the first 5 bits and reverse the order
            xor_results.append(xor_result)
            circuit.reset(qr)
        return xor_results
 

binary_string = create_binary_string()
xor_results1 = simon_problem(binary_string)
 
# Print XORed values, transposed values, and values after the second XOR
for i in range(32):
    print(f"Input: {format(i, '05b')} XOR {fixed_value} = {xor_results1[i]} (After Transposition [E(x) + P(x)] )")

Input: 00000 XOR 10101 = 01000 (After Transposition [E(x) + P(x)] )
Input: 00001 XOR 10101 = 01010 (After Transposition [E(x) + P(x)] )
Input: 00010 XOR 10101 = 00101 (After Transposition [E(x) + P(x)] )
Input: 00011 XOR 10101 = 00111 (After Transposition [E(x) + P(x)] )
Input: 00100 XOR 10101 = 01001 (After Transposition [E(x) + P(x)] )
Input: 00101 XOR 10101 = 01011 (After Transposition [E(x) + P(x)] )
Input: 00110 XOR 10101 = 00101 (After Transposition [E(x) + P(x)] )
Input: 00111 XOR 10101 = 00111 (After Transposition [E(x) + P(x)] )
Input: 01000 XOR 10101 = 11001 (After Transposition [E(x) + P(x)] )
Input: 01001 XOR 10101 = 11001 (After Transposition [E(x) + P(x)] )
Input: 01010 XOR 10101 = 10100 (After Transposition [E(x) + P(x)] )
Input: 01011 XOR 10101 = 10100 (After Transposition [E(x) + P(x)] )
Input: 01100 XOR 10101 = 11010 (After Transposition [E(x) + P(x)] )
Input: 01101 XOR 10101 = 11010 (After Transposition [E(x) + P(x)] )
Input: 01110 XOR 10101 = 10110 (After Transposit